<center><h1>ATOMIC MODEL MRC-VIEWERTOOL</h1></center>
<center><h3>version 1.0</h3> </center>

<center><h5>TU Munich, Dietzlab</h5></center>
<center><h5>author: Elija Feigl, elija.feigl@tum.de</h5></center>
<center><h5>license: GPL-3.0</h5></center>
<center><h5>please cite: ****unpublished***</h5></center>

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
#%gui qt

In [2]:
%matplotlib inline
%gui qt
import linkerapp as linkerAPP
import linkerappUtil as Util

## 1. select project folder and files
select working directoy and required files by executing each cell
each cell opens a seperate FileDialog Popup

In [4]:
# select a working directory
wd = Util.gui_folderDialog()

In [5]:
# select a cadnano deisgn file
json = Util.gui_fileDialog(wd=wd, filter="caDNAno (*.json)")

In [6]:
psf = Util.gui_fileDialog(wd=wd, filter="topology (*.psf)")

In [7]:
pdb = Util.gui_fileDialog(wd=wd, filter="coordinates (*.dcd *.pdb)")

In [8]:
mrc = Util.gui_fileDialog(wd=wd, filter="cryo-map (*.mrc)")

In [9]:
seq = Util.gui_fileDialog(wd=wd, filter="sequence file (*.seq *.txt)")

In [3]:
from pathlib import Path
wd = "/Users/elija/Desktop"
json=Path("/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.json")
psf=Path("/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.psf")
pdb=Path("/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.pdb")
mrc=Path("/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.mrc")
seq=Path("/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.seq")

In [12]:
Files(json=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.json'), psf=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.psf'), coor=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.dcd'), mrc=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.mrc'), seq=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.seq'))

Files(json=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.json'), psf=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.psf'), coor=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.dcd'), mrc=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.mrc'), seq=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__bulletv7/a-iii-e-bulletv7-013__HB_final0-localres/bulletv7.seq'))

# 2. parse filepaths and create Viewer instance
The viewer class contains both the data instances (design, map, model) and the selection functionality.
On creating the linkage between these fit and design is computed. this step might require a couple of seconds, depending on the design size.

In [16]:
files = linkerAPP.Files(json, psf, pdb, mrc, seq)
viewer = linkerAPP.Viewer(wd, files=files)

# 3. select subset of the atomic model
the viewer class supports different selection options:
* design-specific selection: subset specified by helix-id and base-position as in caDNAno designfile.
    * using selection widget -> option A
    * manual selection -> option B
* fit-specific selection: subset directly by MDAnalysis atomselection functionalities
    * example strand-specific selection: -> option C
    
    
In the end, all selections are translated into a selection of atom, stored as an MDAnalysis AtomGroup.   


### Option A: selection with widget 
selection: selected helices (clicked) are displayed in dark grey. the "base" slider selects the base-position range.
context: slider for the area surrounding a selected atom for mrc-cropping (in Å)

<br />

---

the content of the widget is evaluated by executing

``helixandbase, context = viewer.eval_sliders(*sliders)`` 
which returns a tuple containing two lists([helix-ids],[base_ids]) and the context value

the list-tuple can be translated into an AtomGroup by executing:
``atoms_selection, color_dict = viewer.select_by_helixandbase(*helixandbase)``

the method returns the AtomGroup ``atoms_selection`` 

---

(it also returns the dictionary ``color_dict`` containing information about staple coloring in the .json file, that can be used to color staples in the ngl_view widget)

In [5]:
sliders = viewer.make_sliders()

Box(children=(VBox(children=(VBox(children=(HBox(children=(ToggleButton(value=False, disabled=True, layout=Lay…

In [14]:
helixandbase, context = viewer.eval_sliders(*sliders)
atoms_selection, color_dict = viewer.select_by_helixandbase(*helixandbase)

### Option B: manual selection
``viewer.select_by_helixandbase(helices, base)`` can also be called directly using two lists, specifying the helix-ids and base positions.

the method can also be applied to multiple subsets of the data, as MDAnalysis AtomGroups can be combined to realize more complex selections. (more details see option C)

In [13]:
## example Option B:
atoms_selection, color_dict = viewer.select_by_helixandbase([12,13], range(40,68))

### option C: Fit secific selection
generate atomselection using mdAnalysis. 
the MDAnalysis baseclass is available via ``viewer.linker.fit.u``

documentation:(https://www.mdanalysis.org/docs/documentation_pages/selections.html)
NOTES:
* AtomGroups are combinable using set-syntax (f.e. selection = selectionA + selectionB)
* AtomGroups can be inverted by subtraction them from the full system accessible via .universe.atoms attribute. this is usefull for creating masks for multibody and local scanning refinement processes.
* (design-specific staple coloring is not compatible with this option)

In [134]:
## example Option C: all double-stranded segments with staples beginning with the sequence ATCG

# select all staples beginning with a specific sequence, via the intersection of all staples with the correct base condition
# start with all segments and filter with additional conditions
ATCG = u.segments
for idx, X in enumerate(["ADE", "THY", "CYT", "GUA"]):
    ATCG = ATCG.segments & u.select_atoms("resname {} and resid {}".format(X, idx+1)).segments
staple_selection  = ATCG.segments.atoms

# select all paired scaffold bases using the Base-Pair list generated from the design in viewer.linker.link
# start with empty group and iteratively add all scaffold bases
scaffold_selection = Util.AtomGroup([],u)
for residue in staple_selection.residues:
    watson_crick_base = viewer.linker.link.Fbp_full.get(residue.resindex, None)
    if watson_crick_base:
        scaffold_selection += u.residues[watson_crick_base].atoms

# combine scaffold and staple atoms to  form the final atom_selection
atom_selection = scaffold_selection + staple_selection

# 3. Cropping and Zoning 
the generated AtomGroup can be used for
* generating a pdb of te subset represented by the AtomGroup
* creating a new mrc-map that only contains data in the vicinity (context) of the selected atoms 

<br />

---
create subset MRC-file

``viewer.writemrc(atoms_selection, wd, name, [context=context, cut_box=True])``

cut_box: if `True`, remove zero-padding from volume

---

create subset pdb-file (chimeraX compatible)

``viewer.writepdb(atoms_selection, wd, name, [single_frame=True, frame=-1, chimeraX=True])``

single_frame: if `False`, create multiframe pdb for full trajectory (multiframe can take a couple of minutes)

frame: frame index to be saved, default -1 = last (mrc always for last frame)

chimeraX: if `True`, also create chimeraX compatible pdb-file ("name_chim.pdb")

In [7]:
#choose output name
name = "slice-1"

In [18]:
#create subset MRC-file
viewer.writemrc(atoms_selection, name, context=context, cut_box=True)

In [19]:
#create subset pdb-file (chimeraX compatible)
viewer.writepdb(atoms_selection, name, single_frame=True, frame=-1, chimeraX=True)

# (4.  display selection)
* isovalue: isosurface threshold
* backbone: true -> only backbone atoms displayed (faster); false -> all
* color_dict: None -> no staple coloring (significantely faster)

use the play button to show a movie of of the fitting or move the cursor to a specific timestep 

In [31]:
#color_dict = None
Util.fit_widget(atoms_selection, atoms_scaffold, atoms_staple, path=path, isolevel=1.35, backbone=False, color_dict=None)


ValueError: you must provide file extension if using file-like object or text content